# Import packages

In [1]:
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.float_format = '{:,.2f}'.format

# Reading data

In [2]:
df = pd.read_csv('F:/data analysis projects/E-commerce Business Transaction/Data file.csv')
df.head()

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,581482,12/9/2019,22485,Set Of 2 Wooden Market Crates,21.47,12,"17,490.00",United Kingdom
1,581475,12/9/2019,22596,Christmas Star Wish List Chalkboard,10.65,36,"13,069.00",United Kingdom
2,581475,12/9/2019,23235,Storage Tin Vintage Leaf,11.53,12,"13,069.00",United Kingdom
3,581475,12/9/2019,23272,Tree T-Light Holder Willie Winkie,10.65,12,"13,069.00",United Kingdom
4,581475,12/9/2019,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,"13,069.00",United Kingdom


In [3]:
df.shape

(536350, 8)

# Data cleaning

In [4]:
df.isnull().sum()

TransactionNo     0
Date              0
ProductNo         0
ProductName       0
Price             0
Quantity          0
CustomerNo       55
Country           0
dtype: int64

In [5]:
df.dropna(inplace=True)
len(df)

536295

In [6]:
df.dtypes

TransactionNo     object
Date              object
ProductNo         object
ProductName       object
Price            float64
Quantity           int64
CustomerNo       float64
Country           object
dtype: object

In [7]:
df['CustomerNo'] = df['CustomerNo'].astype(int)
df['CustomerNo'] = df['CustomerNo'].astype(str)
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,TransactionNo,Date,ProductNo,ProductName,Price,Quantity,CustomerNo,Country
0,581482,2019-12-09,22485,Set Of 2 Wooden Market Crates,21.47,12,17490,United Kingdom
1,581475,2019-12-09,22596,Christmas Star Wish List Chalkboard,10.65,36,13069,United Kingdom
2,581475,2019-12-09,23235,Storage Tin Vintage Leaf,11.53,12,13069,United Kingdom
3,581475,2019-12-09,23272,Tree T-Light Holder Willie Winkie,10.65,12,13069,United Kingdom
4,581475,2019-12-09,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,13069,United Kingdom


In [8]:
df.describe()

,Price,Quantity
count,"536,295.00","536,295.00"
mean,12.66,9.92
std,8.49,216.67
min,5.13,"-80,995.00"
25%,10.99,1.00
50%,11.94,3.00
75%,14.09,10.00
max,660.62,"80,995.00"


* Negative values in quantity column means (cancelled order due to out of stock condition)

In [9]:
len(df[df['Quantity'] < 0])

8531

In [10]:
cancelled_orders = df[df['Quantity'] < 0]

df.drop(df['Quantity'].index[df['Quantity'] < 0].to_list(), inplace=True)
len(df)

527764

In [11]:
df.describe()

,Price,Quantity
count,"527,764.00","527,764.00"
mean,12.63,10.59
std,7.93,156.79
min,5.13,1.00
25%,10.99,1.00
50%,11.94,3.00
75%,14.09,11.00
max,660.62,"80,995.00"


In [12]:
df['Date'].describe()

count                  527764
unique                    305
top       2019-12-05 00:00:00
freq                     5276
first     2018-12-01 00:00:00
last      2019-12-09 00:00:00
Name: Date, dtype: object

In [13]:
df['TransactionNo'].str.contains('C').sum()
# make sure that dataframe does not contain cancelled transactions

0

* TransactionNo (categorical): a six-digit unique number that defines each transaction. The letter “C” in the code indicates a cancellation.

# Data preparation

In [14]:
df.insert(6, 'Revenue(£)', df['Price'] * df['Quantity'])
df.insert(2, 'Month', df['Date'].dt.month)
df.head()

,TransactionNo,Date,Month,ProductNo,ProductName,Price,Quantity,Revenue(£),CustomerNo,Country
0,581482,2019-12-09,12,22485,Set Of 2 Wooden Market Crates,21.47,12,257.64,17490,United Kingdom
1,581475,2019-12-09,12,22596,Christmas Star Wish List Chalkboard,10.65,36,383.40,13069,United Kingdom
2,581475,2019-12-09,12,23235,Storage Tin Vintage Leaf,11.53,12,138.36,13069,United Kingdom
3,581475,2019-12-09,12,23272,Tree T-Light Holder Willie Winkie,10.65,12,127.80,13069,United Kingdom
4,581475,2019-12-09,12,23239,Set Of 4 Knick Knack Tins Poppies,11.94,6,71.64,13069,United Kingdom


In [15]:
products_frequency = pd.DataFrame()

products_frequency['ProductName'] = df['ProductName'].value_counts().keys()
products_frequency['Frequency'] = df['ProductName'].value_counts().values

products_frequency.head()


,ProductName,Frequency
0,Cream Hanging Heart T-Light Holder,2336
1,Jumbo Bag Red Retrospot,2115
2,Regency Cakestand 3 Tier,2019
3,Party Bunting,1708
4,Lunch Bag Red Retrospot,1597


In [16]:
transactions_frequency = pd.DataFrame()

transactions_frequency['TransactionNo'] = df['TransactionNo'].value_counts().keys()
transactions_frequency['Frequency'] = df['TransactionNo'].value_counts().values

transactions_frequency.head()

,TransactionNo,Frequency
0,573585,1111
1,581219,747
2,581492,730
3,580729,720
4,558475,704


In [29]:
df['Date'] = pd.to_datetime(df['Date']).dt.date
cancelled_orders['Date'] = pd.to_datetime(cancelled_orders['Date']).dt.date

# Save data to be visualized

In [40]:
with pd.ExcelWriter('C:/Users/LENOVO.SXS011/Downloads/Documents/visualiz data.xlsx') as writer:
    df.to_excel(writer, sheet_name='Row data', index=False)
    cancelled_orders.to_excel(writer, sheet_name='Cancelled transactions', index=False)
    products_frequency.to_excel(writer, sheet_name='Products frequency', index=False)
    transactions_frequency.to_excel(writer, sheet_name='Transactions frequency', index=False)